In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
train = pd.concat([train.drop(['spacegroup'], axis=1), 
                   pd.get_dummies(train['spacegroup'], prefix='SG')], axis=1)
test = pd.concat([test.drop(['spacegroup'], axis=1), 
                   pd.get_dummies(test['spacegroup'], prefix='SG')], axis=1)
for c in train.columns:
    if(c.find('angle')!=-1):
        print(c)
        train[c] = np.radians(train[c])
        test[c] = np.radians(test[c])

In [ ]:
def FE(data):
    v = pd.DataFrame()
    v["i0"] = (((np.log(((((data["lattice_vector_3_ang"]) + (data["percent_atom_in"]))/2.0)))) / (((((((((data["percent_atom_ga"]) * 2.0)) * 2.0)) * (((((data["percent_atom_ga"]) * 2.0)) * 2.0)))) + ((9.55804252624511719))))))
    v["i1"] = (((((data["percent_atom_in"]) / (np.sin((data["lattice_angle_gamma_degree"]))))) / (((np.exp((((np.exp((data["percent_atom_in"]))) * (np.exp((data["percent_atom_in"]))))))) + (data["lattice_angle_gamma_degree"])))))
    return v.sum(axis=1).values

def BE(data):
    v = pd.DataFrame()
    v["i0"] = (((data["percent_atom_al"]) + (((((np.exp(((((-((data["fe"])))) / ((((data["percent_atom_ga"]) + (1.071430))/2.0)))))) * (np.exp(((((-((data["fe"])))) / ((((data["percent_atom_ga"]) + (1.071430))/2.0)))))))) / (np.sqrt((np.cos((np.cos((np.sqrt(((((((((np.sin((data["number_of_total_atoms"]))) * (data["lattice_vector_3_ang"]))) * (((np.sin((data["number_of_total_atoms"]))) * (data["lattice_vector_3_ang"]))))) + (((data["percent_atom_al"]) / (data["fe"]))))/2.0)))))))))))))) 
    v["i1"] = (((((2.318180) + (np.sin((data["number_of_total_atoms"]))))) / (((data["lattice_vector_1_ang"]) / ((((((np.sin((((data["number_of_total_atoms"]) / (data["lattice_vector_1_ang"]))))) / (2.318180))) + (np.sin((((0.338235) * (((data["lattice_vector_1_ang"]) / (np.sin((((data["number_of_total_atoms"]) / (np.sin((data["number_of_total_atoms"]))))))))))))))/2.0))))))
    return v.sum(axis=1).values

In [ ]:
print(mean_squared_error(np.log1p(train.formation_energy_ev_natom.values),FE(train)))
train['fe'] = FE(train)
test['fe'] = FE(test)
print(mean_squared_error(np.log1p(train.bandgap_energy_ev.values),BE(train)))

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['formation_energy_ev_natom'] = np.expm1(FE(test))
sub['bandgap_energy_ev'] = np.expm1(BE(test))
sub.to_csv('gpsubmission.csv', index=False)